In [ ]:
from model_definition import model_initializer, solve, pprint

import pyomo.environ as pyo

import matplotlib.pyplot as plt
# Increase figure DPI
plt.rcParams['figure.dpi'] = 300

import numpy as np
from tqdm.auto import tqdm

from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition

# One, Two, Three Model Optimization

## One-Model Optimization

In [ ]:
model = model_initializer()

### Choose the most active nanoparticle catalyst

In [ ]:
model.catalyst_type.fix(2) #Pt
model.nanoparticle_diameter.fix(3) #nm
model.temperature.fix(400) #K
model.pressure.fix(100) #atm

### Maximize the process yield

#### Maximize amount of catalyst

In [ ]:
model.obj = pyo.Objective(expr=model.catalyst_amount, sense=pyo.maximize)
solve(model)
pprint(model)

model.catalyst_amount.fix(model.catalyst_amount.value)

In [ ]:
model.pellet_effectiveness_factor.fix(1)
#model.temperature.fix(400)
#model.pressure.fix(100)
model.cstr_volume.fix(2)

### Optimize for cost

In [ ]:
model.obj = pyo.Objective(expr=model.cost, sense=pyo.minimize)
solve(model)
pprint(model)

In [ ]:
one_model_opt_cost = model.cost.value

print(f"Final cost: ${model.cost.value:.2f} / kg H2")

## Two-Model Optimization

In [ ]:
model = model_initializer()

### Choose the most active nanoparticle catalyst

In [ ]:
model.catalyst_type.fix(2) #Pt
model.nanoparticle_diameter.fix(3) #nm
#model.temperature.fix(400) #K
#model.pressure.fix(100) #atm

### Minimize cost

In [ ]:
solve(model)
pprint(model)

In [ ]:
two_model_opt_cost = model.cost.value

print(f"Final cost: ${model.cost.value:.2f} / kg H2")

## Three-Model Optimization

In [ ]:
model = model_initializer()

In [ ]:
solve(model)
pprint(model)

In [ ]:
three_model_opt_cost = model.cost.value

print(f"Final cost: ${model.cost.value:.2f} / kg H2")

## Final Results

In [ ]:
plt.bar([1, 2, 3], [one_model_opt_cost, two_model_opt_cost, three_model_opt_cost], color = '#ed1111')
plt.xticks([1, 2, 3], ["One-Model", "Two-Model", "Three-Model"])
plt.ylabel("Cost ($ / kg H₂)")

plt.gca().bar_label(plt.gca().containers[0], fmt="$%.2f")

plt.show()

# Economy of Scale Effect

In [ ]:
Pt_costs = []
Pd_costs = []
Cu_costs = []

capacity_list = np.linspace(100, 1500, 30)
solver = SolverFactory('glpk')

model = model_initializer()

model.catalyst_type.fix(2) #Pt
for capacity in tqdm(capacity_list):
    model.station_capacity.fix(capacity)
    solution = solver.solve(model, tee = False)
    if solution.solver.termination_condition is TerminationCondition.optimal:
        Pt_costs.append(model.cost.value)
    else:
        Pt_costs.append(None)

model.catalyst_type.fix(0) #Pd
for capacity in tqdm(capacity_list):
    model.station_capacity.fix(capacity)
    solution = solver.solve(model, tee = False)
    if solution.solver.termination_condition == TerminationCondition.optimal:
        Pd_costs.append(model.cost.value)
    else:
        Pd_costs.append(None)
    Pd_costs = Pd_costs

model.catalyst_type.fix(1) #Cu
for capacity in tqdm(capacity_list):
    model.station_capacity.fix(capacity)
    solution = solver.solve(model, tee = False)
    if solution.solver.termination_condition == TerminationCondition.optimal:
        Cu_costs.append(model.cost.value)
    else:
        Cu_costs.append(None)

In [ ]:
plt.scatter(capacity_list, Pt_costs, label="Pt", color="red")
plt.scatter(capacity_list, Pd_costs, label="Pd", color="blue")
plt.scatter(capacity_list, Cu_costs, label="Cu", color="green")
plt.xlim(0, 1600)
plt.ylim(0, 70)
plt.xlabel("Station Capacity (kg H₂ / day)")
plt.ylabel("Cost ($ / kg H₂)")
plt.grid()
plt.legend()
plt.show()